In [11]:
from classroom import *
import io
from googleapiclient.http import MediaIoBaseDownload
import datetime
from datetime import timedelta
import configparser
import json

In [12]:
#подключение к гугл диску для загрузки файлов
#код взят из доков к апи

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly',
         'https://www.googleapis.com/auth/drive']
drive_service=None

def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token_drive.pickle'):
        with open('token_drive.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'auth/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token_drive.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('drive', 'v3', credentials=creds)

drive_service=main()

#### Загрузка работ из Classroom

In [20]:
#парсинг конфига (в конфиге отметка времени последнего срабатывания грейдера)
try:
    config = configparser.ConfigParser()
    config.read('config.ini')
    last_check_timestamp = config['CONFIG']['last_check_timestamp']
    last_check_timestamp = datetime.datetime.strptime(last_check_timestamp, "%d-%m-%Y %H:%M:%S")
except:
    print("Проблема с конфиг-файлом или парсингом времени")

start=datetime.datetime.now()-timedelta(hours=3) #фиксируем время, в которое начали загружать данные

course = get_course('39405675985') #загружаем курс
course['courseWork'] = get_courseWork(course) #загружаем все задания курса
course['students'] = get_students(course) #загружаем всех студентов курса

#для каждого задания
for task in course['courseWork']:
    submissions = get_submissions(course, task) #загружаем список (всех) работ студентов
    task['studentSubmissions'] = [] #сюда скопируем только те работы, которые еще не проверялись
    
    for submission in submissions: #для каждой работы
        if 'stateHistory' in submission['submissionHistory'][-1]: #если последнее состояние - "работа сдана"
            if 'attachments' in submission['assignmentSubmission']:
                if submission['submissionHistory'][-1]['stateHistory']['state'] == 'TURNED_IN':
                    #если работа была обновлена ПОСЛЕ последнего включения грейдера, оставляем работу (для проверки)
                    if (last_check_timestamp - to_timestamp(submission['updateTime'])).total_seconds() < 0:
                        task['studentSubmissions'].append(submission)

#обновляем время последнего включения грейдера в конфиге
config['CONFIG']['last_check_timestamp'] = start.strftime("%d-%m-%Y %H:%M:%S")
with open('config.ini', 'w') as configfile:
    config.write(configfile)
configfile.close()


list_of_submissions = [] #список загруженных для проверки работ
list_of_errors = [] #список работ, при загрузки которых произошла ошибка

#для каждого задания в курсе
for task in course['courseWork']:
    #для каждой работы, которую нужно проверить
    for subm in task['studentSubmissions']:
        try:
        #код для загрузки файла не мой, че он делает - хз
        ######
            file_id = subm['assignmentSubmission']['attachments'][0]['driveFile']['id'] #id файла на диске
            request = drive_service.files().get_media(fileId=file_id)
            
            #название, под которым будет сохранен файл: <id задания> - <id работы> (без кавычек)
            filename = "for_grading/{} - {}.{}".format(task['id'], 
                                           subm['id'], 
                                           subm['assignmentSubmission']['attachments'][0]['driveFile']['title'].split('.')[-1])
            
            fh = io.FileIO(filename, 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
        ######
            
            
            #если файл успешно загрузился, добавляем запись о нем в файл
            list_of_submissions.append({'courseWorkId': task['id'], 
                                        'courseWorkName': task['title'],
                                       'submissionId': subm['id'], 
                                       'userId': subm['userId'], 
                                       'email': find_student(subm['userId'], course['students'])['emailAddress'],
                                       'firstName': find_student(subm['userId'], course['students'])['firstName'],
                                       'lastName': find_student(subm['userId'], course['students'])['lastName'],
                                       'grade':0})
        except:
            #возникшие проблемы записываются в лог
            logger.warning('проблема с загрузкой: {}, {}'.format(task['title'],
                                                   find_student(subm['userId'], course['students'])['emailAddress']))

print('Загружено файлов: {} - {}'.format(last_check_timestamp, len(list_of_submissions)))
logger.info('Загружено файлов: {}'.format(len(list_of_submissions)))

#файл с инфой о работах, которые нужно проверить
with open("submissions.json", "w") as file:
    file.write(json.dumps({'data':list_of_submissions}, indent = 4))
file.close()

Загружено файлов: 2020-04-13 20:16:37 - 2


#### Выгрузка оценок в Classroom

Грейдер делает свое дело, а именно - достает оценки из файлов студентов и записывает их в поле grade в файл `submissions.json`

In [21]:
#читаем submissions.json
with open("submissions.json", "r") as read_file:
    data = json.load(read_file)['data']

Инфа в файле выглядит следующим образом (когда в файл загружаются данные из классрума, в поле grade стоит 0; грейдер должен проставить в них актуальную оценку):

In [27]:
data

[{'courseWorkId': '83519916191',
  'courseWorkName': 'test3',
  'submissionId': 'Cg4I36OlmcoBEJ-Rs5G3Ag',
  'userId': '108574576705829127319',
  'email': 'vakiryanova@edu.hse.ru',
  'firstName': 'Groups',
  'lastName': 'Technical Account',
  'grade': 0},
 {'courseWorkId': '75649994456',
  'courseWorkName': 'test',
  'submissionId': 'Cg4I36OlmcoBENid3eiZAg',
  'userId': '108574576705829127319',
  'email': 'vakiryanova@edu.hse.ru',
  'firstName': 'Groups',
  'lastName': 'Technical Account',
  'grade': 0}]

Вот тут проблемес с тем, что через апи получается ставить оценки лишь по тем заданиям, что были созданы через апи (а не ручками).  

Но по идее этой проблемы не будет, если работать с сервисного акка (в любом случае это проблема с моей стороны)

In [24]:
result=None
courseId = '39405675985'

#для каждой проверенной работы выставляем оценку в Classroom
for subm in data:
    result = service.courses().courseWork().studentSubmissions().patch(courseId=courseId, 
                                                                    courseWorkId=subm['courseWorkId'], 
                                                                    id=subm['submissionId'], 
                                                                    updateMask='assignedGrade,draftGrade', 
                                                                    body={"draftGrade": 
                                                                          subm['grade'], "assignedGrade": subm['grade']}).execute()
    print(subm['courseWorkName'])

test3


HttpError: <HttpError 403 when requesting https://classroom.googleapis.com/v1/courses/39405675985/courseWork/75649994456/studentSubmissions/Cg4I36OlmcoBENid3eiZAg?updateMask=assignedGrade%2CdraftGrade&alt=json returned "@ProjectPermissionDenied The Developer Console project is not permitted to make this request.">